In [ ]:
## Mount = enables file access in explorer
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re
import numpy as np
import os
from tqdm import tqdm


#Installing required library for getting location:
!pip install GeoText
from geotext import GeoText

#Installing required library for language detection:
#Need >= 3.0.0
!pip install googletrans==3.1.0a0
from googletrans import Translator

#Installing the required libraries for extracting twitter data
!pip install gender-guesser
#Need >= 4.8.0
!pip install tweepy==4.8.0


import tweepy as tw
from tweepy.errors import TooManyRequests

#from datetime import datetime, timedelta
import gender_guesser.detector
import copy
from datetime import date, timedelta, datetime
import warnings
import time

#Library needed for language detection
! pip install langdetect
from langdetect import detect, detect_langs, LangDetectException

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.4 MB 11.0 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=0f4b5acf2b3538d2b9be6b0f3f719129b730d7afe12fb2af877d8fc6c1128e5a
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████

# Pre-processing

## Reading concatenated data

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Bot detection/Data/main_data.json')
df

,created_at_1,user_id,user_name,user_screen_name,user_location,user_description,user_url,protected,followers_count,friends_count,...,notifications,translator_type,pinned_tweet_id,withheld,Bot,Source/label,has_extended_profile,profile_location,default_profile,default_profile_image
0,2020-01-16 02:02:55+00:00,u1217628182611927040,Boaz Barak,boazbaraktcs,"Cambridge, MA",Theoretical Computer Scientist. See also https...,https://t.co/BoMip9FF17,False,7316,215,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
1,2014-07-02 17:56:46+00:00,u2664730894,olawale 💨,wale_io,🎈,creative _,,False,123,1090,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
2,2020-05-30 12:10:45+00:00,u1266703520205549568,panagiota_.b,b_panagiota,None,👽,,False,3,62,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
3,2019-01-26 13:52:49+00:00,u1089159225148882949,"Jacqueline Hodges, MD MPH",jachodges_md,"Charlottesville, VA",mama to maya. ABIM research pathway fellow @UV...,,False,350,577,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
4,2009-04-30 19:01:42+00:00,u36741729,Matthew Stubblefield,Matthew_Brody,United States,Father / SWT Alumnus / Longhorn Fan,,True,240,297,...,To be populated,To be populated,None,None,1,TwiBot22,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058411,"""Fri Mar 02 22:13:07 +0000 2018""",188645930,"""Bessie Gate""","""ikumi0926""","""Hawaii""",To be populated,null,false,24,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058412,"""Sat Mar 10 04:52:16 +0000 2018""",552236544,"""Paula Macey""","""MingzeLi123""","""""",To be populated,null,false,7,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058413,"""Fri Mar 16 19:28:34 +0000 2018""",1933105668,"""Wendy Palmer""","""HeyyJessica""","""""",To be populated,null,false,74,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058414,"""Sat Mar 10 04:42:55 +0000 2018""",3222769956,"""Irene Miln""","""Clown327426514""","""""",To be populated,null,false,3,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None


## changing true/false to True/False, air quotes, and handling variations of null values

In [ ]:

"""
remove_bol_variants:
  Casting strings that represent boolean values to their corresponding boolean values

remove_na_variants:
  Variants of missing data are handled and casted to np.nan

remove_air_qoutes:
  Strings starting and ending with \" are handled

df.user_id = df.user_id.apply(lambda x: int(x[1:]) if type(x) == str else x)
  Some user_ids have u before the id, removing the u and casting to int in these cases
"""


na_lst = ["", '""', "To be populated", "none", "null", 'None ']
for col in tqdm(df.columns.tolist()):

  #Removing boolean variants - excluding user_name, user_location, and user_description as this is text data that will be embedded
  if col not in ['user_name', 'user_location', 'user_description']:
    df[col] = df[col].map(lambda x: x if type(x) != str else False if x.lower().strip() == "false" else True if x.lower().strip() == "true" else x)

  #Removing na variants
  df[col] = df[col].map(lambda x: x if type(x) != str else np.nan if x in na_lst else x)
  
  #Removing air qoutes from the string columns
  df[col] = df[col].map(lambda x: x if type(x) != str else x.lstrip('"').rstrip('"'))
  
  
#Some user_ids have u before the id, removing the u and casting to int in these cases
df.user_id = df.user_id.apply(lambda x: int(x[1:]) if type(x) == str else x)

df

100%|██████████| 45/45 [01:11<00:00,  1.59s/it]


,created_at_1,user_id,user_name,user_screen_name,user_location,user_description,user_url,protected,followers_count,friends_count,...,notifications,translator_type,pinned_tweet_id,withheld,Bot,Source/label,has_extended_profile,profile_location,default_profile,default_profile_image
0,2020-01-16 02:02:55,1217628182611927040,Boaz Barak,boazbaraktcs,Cambridge,Theoretical Computer Scientist. See also https...,https://t.co/BoMip9FF17,False,7316,215,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
1,2014-07-02 17:56:46,2664730894,olawale 💨,wale_io,🎈,creative _,NaN,False,123,1090,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
2,2020-05-30 12:10:45,1266703520205549568,panagiota_.b,b_panagiota,None,👽,NaN,False,3,62,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
3,2019-01-26 13:52:49,1089159225148882949,"Jacqueline Hodges, MD MPH",jachodges_md,Charlottesville,mama to maya. ABIM research pathway fellow @UV...,NaN,False,350,577,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
4,2009-04-30 19:01:42,36741729,Matthew Stubblefield,Matthew_Brody,United States,Father / SWT Alumnus / Longhorn Fan,NaN,True,240,297,...,NaN,NaN,NaN,None,1,TwiBot22,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058411,Fri Mar 02 22:13:07 +0000 2018,188645930,Bessie Gate,ikumi0926,Hawaii,NaN,NaN,False,24,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058412,Sat Mar 10 04:52:16 +0000 2018,552236544,Paula Macey,MingzeLi123,NaN,NaN,NaN,False,7,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058413,Fri Mar 16 19:28:34 +0000 2018,1933105668,Wendy Palmer,HeyyJessica,NaN,NaN,NaN,False,74,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058414,Sat Mar 10 04:42:55 +0000 2018,3222769956,Irene Miln,Clown327426514,NaN,NaN,NaN,False,3,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None


## Trying to identify location - using GeoText

In [ ]:
#Resulting format of user_location: Country, city, city, city
  #Successful for ~ 72% of entries

df["user_location"] = df.user_location.map(lambda x: x if type(x) != str else x if ", ".join(GeoText(x).countries + GeoText(x).cities) == "" else ", ".join(GeoText(x).countries + GeoText(x).cities))
df

,created_at_1,user_id,user_name,user_screen_name,user_location,user_description,user_url,protected,followers_count,friends_count,...,notifications,translator_type,pinned_tweet_id,withheld,Bot,Source/label,has_extended_profile,profile_location,default_profile,default_profile_image
0,2020-01-16 02:02:55+00:00,u1217628182611927040,Boaz Barak,boazbaraktcs,Cambridge,Theoretical Computer Scientist. See also https...,https://t.co/BoMip9FF17,False,7316,215,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
1,2014-07-02 17:56:46+00:00,u2664730894,olawale 💨,wale_io,🎈,creative _,,False,123,1090,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
2,2020-05-30 12:10:45+00:00,u1266703520205549568,panagiota_.b,b_panagiota,None,👽,,False,3,62,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
3,2019-01-26 13:52:49+00:00,u1089159225148882949,"Jacqueline Hodges, MD MPH",jachodges_md,Charlottesville,mama to maya. ABIM research pathway fellow @UV...,,False,350,577,...,To be populated,To be populated,None,None,0,TwiBot22,None,None,None,None
4,2009-04-30 19:01:42+00:00,u36741729,Matthew Stubblefield,Matthew_Brody,United States,Father / SWT Alumnus / Longhorn Fan,,True,240,297,...,To be populated,To be populated,None,None,1,TwiBot22,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058411,"""Fri Mar 02 22:13:07 +0000 2018""",188645930,"""Bessie Gate""","""ikumi0926""","""Hawaii""",To be populated,null,false,24,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058412,"""Sat Mar 10 04:52:16 +0000 2018""",552236544,"""Paula Macey""","""MingzeLi123""","""""",To be populated,null,false,7,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058413,"""Fri Mar 16 19:28:34 +0000 2018""",1933105668,"""Wendy Palmer""","""HeyyJessica""","""""",To be populated,null,false,74,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None
1058414,"""Sat Mar 10 04:42:55 +0000 2018""",3222769956,"""Irene Miln""","""Clown327426514""","""""",To be populated,null,false,3,0,...,false,"""none""",None,None,1,pronbots_2019_df,None,None,None,None


## Fixing the date column

In [ ]:
"""
The date column (created_at_1) comes in different formats depending on the dataset
  Twibot 22 - '%Y-%m-%d %H:%M:%S + %f'
  Good bots - int(x)/ 1e3
  Twibot 20 - '%a %b %d %H:%M:%S +%f %Y '
    Note the white space at the end
  cresci stock 2018 - '%a %b %d %H:%M:%S +%f %Y'
  pronbots 2019 - '%a %b %d %H:%M:%S +%f %Y'
  celebrity 2019 - '%a %b %d %H:%M:%S +%f %Y'
  political bots 2019 - '%a %b %d %H:%M:%S +%f %Y'
  vendor purchased 2019 - '%a %b %d %H:%M:%S +%f %Y'
  verified 2019 - '%a %b %d %H:%M:%S +%f %Y'
  botwiki 2019 - '%a %b %d %H:%M:%S +%f %Y'

"""

date_lst = []


for i in tqdm(range(len(df))):
  try:
    date_lst.append(datetime.strptime(df.created_at_1.iloc[i].split('+')[0], '%Y-%m-%d %H:%M:%S'))
    continue
  except:
    try:
      date_lst.append(datetime.fromtimestamp(int(df.created_at_1.iloc[i]) / 1e3))
      continue
    except:
      try:
        date_lst.append(datetime.strptime(df.created_at_1.iloc[i], '%a %b %d %H:%M:%S +%f %Y'))
      except:
        try:
          date_lst.append(datetime.strptime(df.created_at_1.iloc[i], '%a %b %d %H:%M:%S +%f %Y '))
        except:
          date_lst.append(df.created_at_1.iloc[i])
          

df.created_at_1 = date_lst
df

100%|██████████| 1058416/1058416 [01:20<00:00, 13137.21it/s]


,created_at_1,user_id,user_name,user_screen_name,user_location,user_description,user_url,protected,followers_count,friends_count,...,notifications,translator_type,pinned_tweet_id,withheld,Bot,Source/label,has_extended_profile,profile_location,default_profile,default_profile_image
0,2020-01-16 02:02:55,1217628182611927040,Boaz Barak,boazbaraktcs,Cambridge,Theoretical Computer Scientist. See also https...,https://t.co/BoMip9FF17,False,7316,215,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
1,2014-07-02 17:56:46,2664730894,olawale 💨,wale_io,🎈,creative _,NaN,False,123,1090,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
2,2020-05-30 12:10:45,1266703520205549568,panagiota_.b,b_panagiota,None,👽,NaN,False,3,62,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
3,2019-01-26 13:52:49,1089159225148882949,"Jacqueline Hodges, MD MPH",jachodges_md,Charlottesville,mama to maya. ABIM research pathway fellow @UV...,NaN,False,350,577,...,NaN,NaN,NaN,None,0,TwiBot22,None,None,None,None
4,2009-04-30 19:01:42,36741729,Matthew Stubblefield,Matthew_Brody,United States,Father / SWT Alumnus / Longhorn Fan,NaN,True,240,297,...,NaN,NaN,NaN,None,1,TwiBot22,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058411,2018-03-02 22:13:07,188645930,Bessie Gate,ikumi0926,Hawaii,NaN,NaN,False,24,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058412,2018-03-10 04:52:16,552236544,Paula Macey,MingzeLi123,NaN,NaN,NaN,False,7,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058413,2018-03-16 19:28:34,1933105668,Wendy Palmer,HeyyJessica,NaN,NaN,NaN,False,74,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None
1058414,2018-03-10 04:42:55,3222769956,Irene Miln,Clown327426514,NaN,NaN,NaN,False,3,0,...,False,none,NaN,None,1,pronbots_2019_df,None,None,None,None


## Exluding non-english accounts - using LangDetect


**Motivation:**

There are too many missing values in lang, using an aproach where I apply langdetect on the user descriptions and user names

In total 186 877 accounts are detected as non-english, of which 19 120 are bots

### Generating non-english detect_langs_identified_as_english_df

#### Looping through the descriptions and user names

In [ ]:
"""
Going through user descriptions and user names to try to identify non-english accounts
As user_screen_name resulted in too many false negatives, i.e. letting through too many non-english accounts, it is excluded 
"""

non_english = []
no_bio = []
for i in tqdm(range(len(df))):

  try: 
    if detect(df.user_description.iloc[i]) == 'en':
      continue
    if detect(df.user_name.iloc[i]) == 'en':
      continue
    non_english.append(i)
  except LangDetectException:
    pass
  except TypeError:
    no_bio.append(i)  
  

  0%|          | 0/1058416 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
  0%|          | 12/1058416 [00:00<2:28:37, 118.69it/s]

successfully stored: 0


  1%|          | 10029/1058416 [01:24<1:57:11, 149.09it/s]

successfully stored: 10000


  2%|▏         | 20016/1058416 [02:45<2:09:59, 133.14it/s]

...

 98%|█████████▊| 1042095/1058416 [2:42:03<00:01, 12060.33it/s]

successfully stored: 1040000


100%|█████████▉| 1053126/1058416 [2:42:04<00:00, 12123.48it/s]

successfully stored: 1050000


100%|██████████| 1058416/1058416 [2:42:04<00:00, 108.84it/s]  


#### langdetect excludes too many entries -> using detect_langs for 10 iterations on the excluded user descriptions

In [ ]:
""""
Looping through non_english and querying the user descriptions again, this time for 10 iterations and using detect_langs
  returns [lang1:prob, lang2:prob]
  if en is detected amongst the retun, detect_langs_identified_as_english is populated with the index
  detect_langs returns variate over iterations

For every 10 000 account the system stores detect_langs_identified_as_english

Need to have non_english and no_bio in memory
"""


#Count is used to control storing intervals
count = 0

#Initiating empty variable that will be stored
detect_langs_identified_as_english = []

#looping through non_english
for i in tqdm(non_english):
  
  count += 1

  #Storing at every 10000 iterations
  if count % 10000 == 0:
    #Creating a series
    detect_langs_identified_as_english_df = pd.DataFrame({'detect_langs_identified_as_english': pd.Series(detect_langs_identified_as_english)})
    detect_langs_identified_as_english_df.to_json(FILE_PATH)
  
  #Performing detect_langs for 10 iterations
  for j in range(10):
    languages = detect_langs(df.user_description.iloc[i])
    
    #Checking for english in the returned language list
    if "en" in str(languages):
      detect_langs_identified_as_english.append(i)
      #If english is found, the second for loop breaks
      break

 78%|███████▊  | 146295/186877 [3:07:31<1:14:41,  9.05it/s]

### Working with non_english and detect_langs_identified_as_english_df: - indexes (identified as non_english) | (might be english)

In [ ]:
#Schema: remove non_english except for those that are bots in detect_langs_identified_as_english_df


#Reading in the sources
non_english =pd.read_json(FOLDER_PATH+'bio_non_english_and_no_bio.json').non_english.tolist()
detect_langs_identified_as_english_df = pd.read_json(FOLDER_PATH+'detect_langs_identified_as_english_df.json')

#Declaring indices
detect_lang_indexes = detect_langs_identified_as_english_df.detect_langs_identified_as_english.tolist()

#subsetting bots from detect_lang_indexes
Bots_in_detect_lang_indexes = df.iloc[detect_lang_indexes][df.Bot ==1].index.tolist()

#removing Bots_in_detect_lang_indexes from non_english
non_english_excluding_Bots_in_detect_lang_indexes = [x for x in non_english if x not in Bots_in_detect_lang_indexes]

#Dropping the unwanted indices
df = df.drop(non_english_excluding_Bots_in_detect_lang_indexes)

#Reseting index
df = df.reset_index(drop=True)

df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


## Dropping duplicate user_ids

In [ ]:
df = df.drop_duplicates(subset='user_id', keep="first")
df

# Storing the pre-processed data

In [ ]:
df.to_json(FILE_PATH)